# WIP!!!

In [ ]:
! nvidia-smi

# inference

In [ ]:
def inference(model, df_inf, tknzr):
    ds_inf = CNNDailyMailDataset(df_inf, tknzr)
    dl_inf = DataLoader(ds_inf, batch_size=Config.batch_size, shuffle=False, num_workers=Config.num_workers)

    model.to(device)
    model.eval()
    
    summary_list = []
    for (input_ids, attention_mask, _, _) in tqdm(dl_inf):
        # set data to same device as model
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # run model
        with torch.no_grad():
            out = model.generate(input_ids, attention_mask=attention_mask,
                               do_sample=True, 
                               top_k=50, 
                               top_p=0.95
                              )
        summaries = tokenizer.batch_decode(out, skip_special_tokens=True)
        summary_list.extend(summaries)

    torch.cuda.empty_cache()
    gc.collect()

    df_generated = df_inf.copy()
    df_generated['generated'] = summary_list
    return df_generated

In [ ]:
df_generated = inference(t5_model, df_test[:100], tokenizer)